In [53]:
import os 
import streamlit
from langchain_groq import ChatGroq
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
from langchain.text_splitter import RecursiveCharacterTextSplitter
import dotenv 
dotenv.load_dotenv()


True

# login hugging face and setup llm

In [54]:

from huggingface_hub import login
llm=ChatGroq(
    groq_api_key=os.getenv('GROQ_API_KEY'),
    model_name='llama-3.1-70b-versatile'
)
login(token=os.getenv('HUGGINGFACEHUB_API_TOKEN'),add_to_git_credential=True)


Token is valid (permission: fineGrained).
Your token has been saved in your configured git credential helpers (manager).
Your token has been saved to C:\Users\26amr\.cache\huggingface\token
Login successful


# split the data into chunks 

In [55]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, load_index_from_storage, StorageContext
from llama_index.vector_stores.faiss import FaissVectorStore
# import faiss
# embeddings=OllamaEmbeddings()
# loader=WebBaseLoader('https://en.wikipedia.org/wiki/Elon_Musk')
# docs=loader.load()
# text_splitter=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200)
# documents=text_splitter.split_documents(docs)
# print(documents)

documents=SimpleDirectoryReader('../data/').load_data()



In [56]:
import faiss
d=1536
faiss_index=faiss.IndexFlatL2(d)
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader 
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings
embed_model=HuggingFaceEmbedding(
    model_name='BAAI/bge-small-en-v1.5'
)
# document_text=[doc.page_content for doc in documents]
# embeddings=[embed_model.embed_query(text) for text in document_text]



In [57]:
from llama_index.core import Settings

Settings.chunk_size=1024
Settings.embed_model=embed_model
Settings.llm=llm

In [ ]:
index=VectorStoreIndex.from_documents(documents=documents,embed_model=Settings.embed_model)
# save the data 
index.storage_context.persist('../data/')
query_engine=index.as_query_engine()

In [62]:
response=query_engine.query('summarize on attorney general ')
response.response

"The Attorney General is mentioned in the provided text as the authority who provides opinions and guidance on various matters. There are two Attorney General's opinions mentioned. \n\nOne opinion is from C.A. Culberson, who is of the opinion that the land department of a railroad company is not considered one of the general offices within the meaning of the statutes. This opinion is based on the history and policy of the legislation and the specific facts of the case.\n\nThe other opinion is from Frank Andrews, Office Assistant Attorney General, who is of the opinion that a judgment against an original land purchaser is void if the purchaser had properly transferred the land and the transfer was on file in the General Land office prior to the institution of the suit. Andrews also provides an opinion on the constitutionality of a law related to the Penitentiary Board's authority to purchase lands for the purpose of establishing State farms and employing convict labor."

# embedding the chunks 

In [ ]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader 
from llama_index.vector_stores.faiss import FaissVectorStore
from llama_index.core import Settings
embed_model=HuggingFaceEmbedding(
    model_name='BAAI/bge-small-en-v1.5'
)
# vector_store=FaissVectorStore.from_documents(documents,embed_model)

# from llama_index import FAISS
# from llaam.embeddings.huggingface import HuggingFaceEmbedding

# # Initialize the embedding model
# embed_model = HuggingFaceEmbedding(model_name='BAAI/bge-small-en-v1.5')

# # Ensure your documents are loaded in a list of text (e.g., `documents` is a list of strings)
# # If your documents are more complex objects, extract their text content first
# document_texts = [doc.page_content for doc in documents]  # Adjust if needed

# # Generate embeddings for each document text
# embeddings = [embed_model.embed_query(text) for text in document_texts]  # Using embed_query here

# # Create the FAISS index from the embeddings
# vectors = FAISS.from_embeddings(embeddings, metadatas=[doc.metadata for doc in documents])

# print("FAISS index created successfully with the HuggingFace embedding model.")


AttributeError: from_documents

In [ ]:
prompt=ChatPromptTemplate.from_template(
    """ 
    You are an experienced immigration lawyer providing legal advice to an immigrant client. Analyze the context and documents carefully, considering all possible steps, risks, and requirements for the client's immigration status. 

<Context>
{context}
</Context>

<Documents>
{documents}
</Documents>

Considerations:
1. Identify and clarify the primary legal issues affecting the client's case.
2. Provide a step-by-step overview of the necessary actions the client should take, considering both short-term and long-term implications.
3. Explain the potential legal outcomes and any significant risks or benefits.
4. Highlight any documents, forms, or evidence the client will need to prepare or retain.
5. Offer any additional advice on maintaining compliance with immigration laws or procedures.

Question: {input}

Please provide a clear, structured response that includes legal options, procedural steps, and any supporting advice for the client.

"""
)

In [ ]:
document_chain=cre

NameError: name 'cre' is not defined